See python-snowflake handout for details.

What you will need to do:

In Snowflake:
- Collect the account information, warehouse name, database name, etc (See Snowflake connection info below)
- Temporarily disable MFA if it is enabled
- Make sure you know the number of rows and the total for a numeric column for testing.

In VSC:
- Create a new file to hold the environment variables (the snowflake data above)

In python notebook:
- Install the snowflake-connector
- Run the code

# Snowflake Connection

- SNOWFLAKE_ACCOUNT=your_account_identifier
- SNOWFLAKE_USER=your_username
- SNOWFLAKE_PASSWORD=your_password
- SNOWFLAKE_WAREHOUSE=your_warehouse_name
- SNOWFLAKE_DATABASE=your_database_name
- SNOWFLAKE_SCHEMA=your_schema_name
- SNOWFLAKE_ROLE=your_role_name

In [14]:
%pip install snowflake-connector-python pyarrow -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import snowflake.connector
from dotenv import load_dotenv
import os
import pandas as pd


In [2]:
# Load environment variables from .env file
load_dotenv()

# Retrieve values
user = os.getenv("SNOWFLAKE_USER")
password = os.getenv("SNOWFLAKE_PASSWORD")
account = os.getenv("SNOWFLAKE_ACCOUNT")
warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
database = os.getenv("SNOWFLAKE_DATABASE")
schema = os.getenv("SNOWFLAKE_SCHEMA")
role = os.getenv("SNOWFLAKE_ROLE")

# user = SNOWFLAKE_USER
# password = SNOWFLAKE_PASSWORD
# account = SNOWFLAKE_ACCOUNT
# warehouse = SNOWFLAKE_WAREHOUSE
# database = SNOWFLAKE_DATABASE
# schema = SNOWFLAKE_SCHEMA
# role = SNOWFLAKE_ROLE

conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    role=role,
)

# Create a cursor object
cur = conn.cursor()

# Execute a query
cur.execute("SELECT count(*) from orders")

# Fetch result
print(cur.fetchone())

# Clean up
cur.close()
conn.close()

(1500000,)


In [3]:
# Create a cursor object
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    role=role,
)
cur = conn.cursor()

# Execute a query and load into DataFrame
cur.execute("SELECT * FROM orders")
result_df = cur.fetch_pandas_all()

# Clean up
cur.close()
conn.close()

result_df

,O_ORDERKEY,O_CUSTKEY,O_ORDERSTATUS,O_TOTALPRICE,O_ORDERDATE,O_ORDERPRIORITY,O_CLERK,O_SHIPPRIORITY,O_COMMENT
0,1200001,121361,F,60106.33,1994-01-24,1-URGENT,Clerk#000000340,0,ourts are carefully above the slyly final theo...
1,1200002,1775,O,194561.08,1996-12-06,2-HIGH,Clerk#000000709,0,ts. ironic sheaves along the
2,1200003,122593,F,10061.57,1994-01-23,2-HIGH,Clerk#000000141,0,bravely final accounts serve. carefully regula...
3,1200004,6394,O,206408.82,1996-06-10,3-MEDIUM,Clerk#000000545,0,posits wake carefully. pinto beans sleep. b
4,1200005,57130,F,234800.62,1995-01-04,2-HIGH,Clerk#000000473,0,tes. fluffily even packages into the regular g...
...,...,...,...,...,...,...,...,...,...
1499995,4199972,135494,F,234876.94,1992-11-20,2-HIGH,Clerk#000000642,0,anent packages around the e
1499996,4199973,97459,F,55190.12,1992-12-25,2-HIGH,Clerk#000000063,0,ully even pinto beans cajole furiously along
1499997,4199974,118453,O,221822.59,1997-06-18,5-LOW,Clerk#000000313,0,s haggle around the even excuses. quic
1499998,4199975,35011,O,134591.43,1996-07-09,3-MEDIUM,Clerk#000000248,0,nal theodolites use furiously unusual asymptot...


In [15]:
# Parameterized queries and efficient fetching helpers
from typing import Dict, Iterable, Optional
import pandas as pd
import snowflake.connector


def _get_conn():
    """Open a fresh Snowflake connection using env vars already loaded above."""
    return snowflake.connector.connect(
        user=user,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema,
        role=role,
    )


def fetch_df(sql: str, params: Optional[Dict] = None) -> pd.DataFrame:
    """
    Run a parameterized query and return a single pandas DataFrame.
    Use for smaller result sets.
    Example:
        df = fetch_df(
            "SELECT COUNT(*) AS row_count FROM orders WHERE O_ORDERDATE >= %(start)s AND O_ORDERDATE < %(end)s",
            {"start": "1996-01-01", "end": "1996-02-01"},
        )
    """
    with _get_conn() as _conn:
        with _conn.cursor() as _cur:
            _cur.execute(sql, params)
            return _cur.fetch_pandas_all()


def fetch_df_batches(
    sql: str,
    params: Optional[Dict] = None,
    batch_size: int = 100_000,
) -> Iterable[pd.DataFrame]:
    """
    Run a parameterized query and yield pandas DataFrames in batches.
    Uses fetch_pandas_batches when available; falls back to fetchmany.
    Example:
        total = 0
        for i, part in enumerate(
            fetch_df_batches("SELECT * FROM orders WHERE O_TOTALPRICE > %(min_total)s", {"min_total": 100}), 1
        ):
            print(f"Batch {i}: {len(part)} rows")
            total += len(part)
        print("Total rows:", total)
    """
    with _get_conn() as _conn:
        with _conn.cursor() as _cur:
            _cur.execute(sql, params)
            # Preferred: use native pandas batches if available (Snowflake Connector >= 3.0)
            if hasattr(_cur, "fetch_pandas_batches"):
                for batch in _cur.fetch_pandas_batches():
                    yield batch
            else:
                # Fallback: build DataFrames from fetchmany
                cols = [d[0] for d in _cur.description]
                while True:
                    rows = _cur.fetchmany(batch_size)
                    if not rows:
                        break
                    yield pd.DataFrame.from_records(rows, columns=cols)


# --- Quick smoke tests (safe to run) ---
# Small param query
try:
    example_small = fetch_df(
        "SELECT COUNT(*) AS row_count FROM orders WHERE O_ORDERDATE >= %(start)s AND O_ORDERDATE < %(end)s",
        {"start": "1996-01-01", "end": "1996-02-01"},
    )
    print(example_small)
except Exception as e:
    print("Example small query skipped:", e)

# Large (batched) example
try:
    total_rows = 0
    for i, df_batch in enumerate(
        fetch_df_batches(
            "SELECT * FROM orders WHERE O_TOTALPRICE > %(min_total)s",
            params={"min_total": 100},
            batch_size=200_000,
        ),
        start=1,
    ):
        print(f"Batch {i}: {len(df_batch)} rows")
        total_rows += len(df_batch)
    print("Total rows fetched:", total_rows)
except Exception as e:
    print("Batched example skipped:", e)


   ROW_COUNT
0      19217
Batch 1: 1096 rows
Batch 2: 4533 rows
Batch 3: 6790 rows
Batch 4: 15912 rows
Batch 1: 1096 rows
Batch 2: 4533 rows
Batch 3: 6790 rows
Batch 4: 15912 rows
Batch 5: 27051 rows
Batch 5: 27051 rows
Batch 6: 57637 rows
Batch 7: 36981 rows
Batch 8: 1094 rows
Batch 9: 4541 rows
Batch 10: 6833 rows
Batch 6: 57637 rows
Batch 7: 36981 rows
Batch 8: 1094 rows
Batch 9: 4541 rows
Batch 10: 6833 rows
Batch 11: 15921 rows
Batch 11: 15921 rows
Batch 12: 27068 rows
Batch 13: 57759 rows
Batch 14: 36784 rows
Batch 15: 1099 rows
Batch 16: 4554 rows
Batch 12: 27068 rows
Batch 13: 57759 rows
Batch 14: 36784 rows
Batch 15: 1099 rows
Batch 16: 4554 rows
Batch 17: 6808 rows
Batch 18: 15929 rows
Batch 19: 26995 rows
Batch 17: 6808 rows
Batch 18: 15929 rows
Batch 19: 26995 rows
Batch 20: 57720 rows
Batch 21: 36895 rows
Batch 22: 1105 rows
Batch 23: 4532 rows
Batch 24: 6841 rows
Batch 20: 57720 rows
Batch 21: 36895 rows
Batch 22: 1105 rows
Batch 23: 4532 rows
Batch 24: 6841 rows
Batch 25

In [17]:
# Stream large query results to disk (Parquet/CSV) without holding all rows in memory
from pathlib import Path
import os
import pandas as pd


def stream_to_parquet_dir(
    sql: str,
    params: dict | None = None,
    out_dir: str = "parquet_out",
    base_name: str = "result",
    batch_size: int = 200_000,
    engine: str = "pyarrow",
    compression: str | None = "snappy",
) -> int:
    """Execute a query and write results to multiple Parquet files in batches.
    Returns the total number of rows written.
    """
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    total = 0
    for i, df_part in enumerate(fetch_df_batches(sql, params=params, batch_size=batch_size), start=1):
        part_path = Path(out_dir) / f"{base_name}_part_{i:05d}.parquet"
        df_part.to_parquet(part_path, engine=engine, index=False, compression=compression)
        total += len(df_part)
        print(f"Wrote {len(df_part)} rows to {part_path}")
    print(f"Total rows written to Parquet: {total}")
    return total


def stream_to_csv_dir(
    sql: str,
    params: dict | None = None,
    out_dir: str = "csv_out",
    base_name: str = "result",
    batch_size: int = 200_000,
    float_format: str | None = None,
) -> int:
    """Execute a query and write results to multiple CSV files in batches.
    Returns the total number of rows written.
    """
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    total = 0
    for i, df_part in enumerate(fetch_df_batches(sql, params=params, batch_size=batch_size), start=1):
        part_path = Path(out_dir) / f"{base_name}_part_{i:05d}.csv"
        df_part.to_csv(part_path, index=False, float_format=float_format)
        total += len(df_part)
        print(f"Wrote {len(df_part)} rows to {part_path}")
    print(f"Total rows written to CSV: {total}")
    return total


# --- Example usage (uncomment to run) ---
# total_csv = stream_to_csv_dir(
#     "SELECT * FROM orders WHERE O_TOTALPRICE > %(min_total)s",
#     params={"min_total": 100},
#     out_dir="csv_orders_over_100",
#     base_name="orders_over_100",
#     batch_size=250_000,
# )

total_parquet = stream_to_parquet_dir(
    "SELECT * FROM orders WHERE O_ORDERDATE >= %(start)s AND O_ORDERDATE < %(end)s",
    params={"start": "1996-01-01", "end": "1996-02-01"},
    out_dir="parquet_orders_jan",
    base_name="orders_jan",
    batch_size=250_000,
)

Wrote 1112 rows to parquet_orders_jan/orders_jan_part_00001.parquet
Wrote 802 rows to parquet_orders_jan/orders_jan_part_00002.parquet
Wrote 1088 rows to parquet_orders_jan/orders_jan_part_00003.parquet
Wrote 887 rows to parquet_orders_jan/orders_jan_part_00004.parquet
Wrote 1104 rows to parquet_orders_jan/orders_jan_part_00005.parquet
Wrote 770 rows to parquet_orders_jan/orders_jan_part_00006.parquet
Wrote 1101 rows to parquet_orders_jan/orders_jan_part_00007.parquet
Wrote 833 rows to parquet_orders_jan/orders_jan_part_00008.parquet
Wrote 1088 rows to parquet_orders_jan/orders_jan_part_00003.parquet
Wrote 887 rows to parquet_orders_jan/orders_jan_part_00004.parquet
Wrote 1104 rows to parquet_orders_jan/orders_jan_part_00005.parquet
Wrote 770 rows to parquet_orders_jan/orders_jan_part_00006.parquet
Wrote 1101 rows to parquet_orders_jan/orders_jan_part_00007.parquet
Wrote 833 rows to parquet_orders_jan/orders_jan_part_00008.parquet
Wrote 1107 rows to parquet_orders_jan/orders_jan_part_0

In [19]:
# Read Parquet parts back into a single DataFrame
from pathlib import Path
import pandas as pd
import pyarrow.dataset as ds


def read_parquet_dir(out_dir: str, base_name: str | None = None, columns: list[str] | None = None) -> pd.DataFrame:
    """
    Read a directory of Parquet part files into a single pandas DataFrame.
    - out_dir: directory containing the Parquet parts
    - base_name: optional prefix filter like "orders_jan" to match files named
      "{base_name}_part_00001.parquet"; if None, reads all Parquet files in the directory
    - columns: optional list of column names to project
    """
    p = Path(out_dir)
    if base_name:
        files = sorted(p.glob(f"{base_name}_part_*.parquet"))
        if not files:
            raise FileNotFoundError(f"No Parquet parts found matching {base_name}_part_*.parquet in {out_dir}")
        dataset = ds.dataset([str(f) for f in files], format="parquet")
    else:
        dataset = ds.dataset(str(p), format="parquet")
    table = dataset.to_table(columns=columns)
    return table.to_pandas()

# --- Example: load the data we just wrote ---
df_orders_jan = read_parquet_dir("parquet_orders_jan", base_name="orders_jan")
print(df_orders_jan.shape)
df_orders_jan.head()

(19217, 9)


,O_ORDERKEY,O_CUSTKEY,O_ORDERSTATUS,O_TOTALPRICE,O_ORDERDATE,O_ORDERPRIORITY,O_CLERK,O_SHIPPRIORITY,O_COMMENT
0,1,36901,O,173665.47,1996-01-02,5-LOW,Clerk#000000951,0,nstructions sleep furiously among
1,7,39136,O,252004.18,1996-01-10,2-HIGH,Clerk#000000470,0,ly special requests
2,1284,133972,O,153911.68,1996-01-07,2-HIGH,Clerk#000000492,0,s. blithely silent deposits s
3,1731,127777,O,317068.68,1996-01-06,1-URGENT,Clerk#000000268,0,"lithely regular, final instructions. ironic, e..."
4,1926,92971,O,149651.08,1996-01-31,2-HIGH,Clerk#000000568,0,cajole. even warhorses sleep carefully.


In [22]:
# Explicit schema → Parquet type mapping with PyArrow
from decimal import Decimal
from typing import Optional, Dict, Iterable
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd


def _infer_decimal_precision_scale(values: Iterable[Decimal]) -> tuple[int, int]:
    """Infer (precision, scale) for a sequence of Decimal values."""
    max_prec = 1
    max_scale = 0
    for v in values:
        if v is None:
            continue
        t = v.as_tuple()
        digits = len(t.digits)
        scale = -t.exponent if t.exponent < 0 else 0
        # If number has leading zeros (e.g., 0.01), precision should at least include integer part
        int_digits = digits - scale
        if int_digits < 1:
            int_digits = 1
        max_prec = max(max_prec, int_digits + scale)
        max_scale = max(max_scale, scale)
    # Cap to Arrow/Snowflake common bounds
    max_prec = min(max_prec, 38)
    max_scale = min(max_scale, 37)
    return max_prec, max_scale


def pandas_series_to_pa_type(s: pd.Series, decimal_override: Optional[pa.DataType] = None) -> pa.DataType:
    """Map a pandas Series to an appropriate PyArrow type, with optional decimal override."""
    if decimal_override is not None:
        return decimal_override
    dt = s.dtype
    if pd.api.types.is_integer_dtype(dt):
        # Choose 64-bit to be safe for IDs
        return pa.int64()
    if pd.api.types.is_bool_dtype(dt):
        return pa.bool_()
    if pd.api.types.is_float_dtype(dt):
        return pa.float64()
    if pd.api.types.is_datetime64_any_dtype(dt):
        # Use nanosecond precision; add tz if present
        tz = getattr(getattr(s, 'dt', None), 'tz', None)
        return pa.timestamp('ns', tz=str(tz)) if tz is not None else pa.timestamp('ns')
    if pd.api.types.is_object_dtype(dt):
        # Try Detect Decimal
        non_null = [v for v in s if v is not None]
        if non_null and all(isinstance(v, Decimal) for v in non_null):
            prec, scale = _infer_decimal_precision_scale(non_null)
            return pa.decimal128(precision=prec, scale=scale)
        # Fallback to string for mixed/unknown object
        return pa.string()
    # Default fallback
    return pa.string()


def build_pa_schema(
    df: pd.DataFrame,
    decimal_overrides: Optional[Dict[str, pa.DataType]] = None,
    explicit_type_overrides: Optional[Dict[str, pa.DataType]] = None,
    preserve_order: bool = True,
 ) -> pa.Schema:
    """Build a PyArrow schema from a pandas DataFrame.
    - decimal_overrides: per-column decimal types (e.g., {'O_TOTALPRICE': pa.decimal128(12,2)})
    - explicit_type_overrides: per-column forced types (any pa.DataType)
    """
    fields: list[pa.Field] = []
    decimal_overrides = decimal_overrides or {}
    explicit_type_overrides = explicit_type_overrides or {}
    cols = df.columns if preserve_order else sorted(df.columns)
    for col in cols:
        if col in explicit_type_overrides:
            pa_type = explicit_type_overrides[col]
        else:
            dec_override = decimal_overrides.get(col)
            pa_type = pandas_series_to_pa_type(df[col], dec_override)
        fields.append(pa.field(col, pa_type, nullable=True))
    return pa.schema(fields)


def write_df_parquet_with_schema(
    df: pd.DataFrame,
    path: str,
    schema: Optional[pa.Schema] = None,
    compression: str = 'snappy',
    coerce_timestamps: Optional[str] = None,
    allow_truncated_timestamps: bool = True,
 ) -> None:
    """Write a pandas DataFrame to Parquet using a fixed PyArrow schema."""
    table = pa.Table.from_pandas(
        df, schema=schema, preserve_index=False,
        safe=True,
    )
    pq.write_table(
        table, path, compression=compression,
        coerce_timestamps=coerce_timestamps,
        allow_truncated_timestamps=allow_truncated_timestamps,
    )


def _quant_for_scale(scale: int) -> Decimal:
    """Return a Decimal quantizer for a given scale (e.g., scale=2 -> Decimal('0.01'))."""
    if scale <= 0:
        return Decimal('1')
    return Decimal('0.' + ('0' * (scale - 1)) + '1')


def coerce_df_to_schema(df: pd.DataFrame, schema: pa.Schema) -> pd.DataFrame:
    """Coerce pandas DataFrame columns to types compatible with the provided Arrow schema.

    - Decimal fields: convert values to Decimal with the correct scale.
    - Date32 fields: convert to Python date objects.
    - Int64 fields: cast to pandas nullable Int64 where possible.
    - String fields: cast to pandas string dtype.
    """
    out = df.copy()
    for field in schema:
        name = field.name
        if name not in out.columns:
            continue
        at = field.type
        s = out[name]
        try:
            if pa.types.is_decimal(at):
                scale = at.scale
                q = _quant_for_scale(scale)
                out[name] = s.apply(lambda x: None if pd.isna(x) else (x if isinstance(x, Decimal) else Decimal(str(x))).quantize(q))
            elif pa.types.is_date32(at) or pa.types.is_date64(at):
                out[name] = pd.to_datetime(s, errors='coerce').dt.date
            elif pa.types.is_integer(at):
                # Best effort to coerce numerics to Int64 (nullable)
                out[name] = pd.to_numeric(s, errors='coerce').astype('Int64')
            elif pa.types.is_string(at):
                out[name] = s.astype('string')
            # Other types fall back to original
        except Exception:
            # If coercion fails, leave the column as-is; Arrow may still handle it
            pass
    return out


def stream_to_parquet_dir_with_schema(
    sql: str,
    params: Optional[Dict] = None,
    out_dir: str = 'parquet_out',
    base_name: str = 'result',
    batch_size: int = 200_000,
    decimal_overrides: Optional[Dict[str, pa.DataType]] = None,
    explicit_type_overrides: Optional[Dict[str, pa.DataType]] = None,
    compression: str = 'snappy',
 ) -> tuple[int, pa.Schema]:
    """Stream query results and write Parquet files using a consistent, explicit schema.
    Returns (total_rows_written, schema_used).
    """
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    total = 0
    schema_used: Optional[pa.Schema] = None
    for i, df_part in enumerate(fetch_df_batches(sql, params=params, batch_size=batch_size), start=1):
        if schema_used is None:
            schema_used = build_pa_schema(
                df_part,
                decimal_overrides=decimal_overrides,
                explicit_type_overrides=explicit_type_overrides,
            )
        # Coerce pandas dtypes to match target Arrow schema (Decimal/Date/etc.)
        df_coerced = coerce_df_to_schema(df_part, schema_used)
        part_path = Path(out_dir) / f"{base_name}_part_{i:05d}.parquet"
        write_df_parquet_with_schema(
            df_coerced, str(part_path), schema=schema_used, compression=compression
        )
        total += len(df_part)
        print(f"Wrote {len(df_part)} rows to {part_path}")
    if schema_used is None:
        # No rows; build empty schema using overrides if provided
        empty_df = pd.DataFrame({})
        schema_used = build_pa_schema(empty_df)
    print(f"Total rows written to Parquet: {total}")
    return total, schema_used


# --- Example (uncomment to run with your environment) ---
decimal_map = {
    "O_TOTALPRICE": pa.decimal128(12, 2),
}
explicit_map = {
    "O_ORDERKEY": pa.int64(),       # NUMBER(38,0)
    "O_CUSTKEY": pa.int64(),        # NUMBER(38,0)
    "O_SHIPPRIORITY": pa.int64(),   # NUMBER(38,0)
    "O_ORDERDATE": pa.date32(),     # DATE → date32
    # VARCHAR columns default to string
}
total, schema_used = stream_to_parquet_dir_with_schema(
    "SELECT O_ORDERKEY, O_CUSTKEY, O_ORDERSTATUS, O_TOTALPRICE, O_ORDERDATE, O_ORDERPRIORITY, O_CLERK, O_SHIPPRIORITY, O_COMMENT FROM orders WHERE O_TOTALPRICE > %(min_total)s",
    params={"min_total": 100},
    out_dir="parquet_orders_schema",
    base_name="orders",
    batch_size=250_000,
    decimal_overrides=decimal_map,
    explicit_type_overrides=explicit_map,
)
print(schema_used)

Wrote 1112 rows to parquet_orders_schema/orders_part_00001.parquet
Wrote 4560 rows to parquet_orders_schema/orders_part_00002.parquet
Wrote 6786 rows to parquet_orders_schema/orders_part_00003.parquet
Wrote 6786 rows to parquet_orders_schema/orders_part_00003.parquet
Wrote 15906 rows to parquet_orders_schema/orders_part_00004.parquet
Wrote 15906 rows to parquet_orders_schema/orders_part_00004.parquet
Wrote 27055 rows to parquet_orders_schema/orders_part_00005.parquet
Wrote 27055 rows to parquet_orders_schema/orders_part_00005.parquet
Wrote 57681 rows to parquet_orders_schema/orders_part_00006.parquet
Wrote 57681 rows to parquet_orders_schema/orders_part_00006.parquet
Wrote 36900 rows to parquet_orders_schema/orders_part_00007.parquet
Wrote 1110 rows to parquet_orders_schema/orders_part_00008.parquet
Wrote 4539 rows to parquet_orders_schema/orders_part_00009.parquet
Wrote 36900 rows to parquet_orders_schema/orders_part_00007.parquet
Wrote 1110 rows to parquet_orders_schema/orders_part_0

(19217, 9)


,O_ORDERKEY,O_CUSTKEY,O_ORDERSTATUS,O_TOTALPRICE,O_ORDERDATE,O_ORDERPRIORITY,O_CLERK,O_SHIPPRIORITY,O_COMMENT
0,1,36901,O,173665.47,1996-01-02,5-LOW,Clerk#000000951,0,nstructions sleep furiously among
1,7,39136,O,252004.18,1996-01-10,2-HIGH,Clerk#000000470,0,ly special requests
2,1284,133972,O,153911.68,1996-01-07,2-HIGH,Clerk#000000492,0,s. blithely silent deposits s
3,1731,127777,O,317068.68,1996-01-06,1-URGENT,Clerk#000000268,0,"lithely regular, final instructions. ironic, e..."
4,1926,92971,O,149651.08,1996-01-31,2-HIGH,Clerk#000000568,0,cajole. even warhorses sleep carefully.
